In [ ]:
import glob
import os
import subprocess

def convert(n, d):
    get_ipython().system("jupyter nbconvert {} --to markdown --output {}".format(n, d))
    subprocess.check_call(["jupyter", "nbconvert", n, "--to", "markdown", "--output", d])
    
for subdir in glob.glob("./sem*"):
    notebooks = glob.glob(subdir + "/*.ipynb")
    print(subdir, notebooks)
    for m in glob.glob(subdir + "/*.md"):
        os.remove(m)
        
#     for n in notebooks:
#         subprocess.check_call(["jupyter", "nbconvert", n, "--to", "pdf", 
#                                "--output", os.path.basename(n.replace(".ipynb", ""))])

    if len(notebooks) == 1:
        convert(notebooks[0], "README")
    else:
        for n in notebooks:
            convert(n, os.path.basename(n.replace(".ipynb", "")))
        


./sem04-asm-arm ['./sem04-asm-arm/arm.ipynb']
[NbConvertApp] Converting notebook ./sem04-asm-arm/arm.ipynb to markdown
[NbConvertApp] Writing 7422 bytes to ./sem04-asm-arm/README.md
./sem11-mmap-instrumentation ['./sem11-mmap-instrumentation/mmap_and_instrumentation.ipynb']
[NbConvertApp] Converting notebook ./sem11-mmap-instrumentation/mmap_and_instrumentation.ipynb to markdown
[NbConvertApp] Writing 23442 bytes to ./sem11-mmap-instrumentation/README.md
./sem10-file-attributes ['./sem10-file-attributes/file-attrib.ipynb']
[NbConvertApp] Converting notebook ./sem10-file-attributes/file-attrib.ipynb to markdown
[NbConvertApp] Writing 16319 bytes to ./sem10-file-attributes/README.md
./sem03-ints-floats ['./sem03-ints-floats/sem3_floats.ipynb', './sem03-ints-floats/sem3_ints.ipynb']
[NbConvertApp] Converting notebook ./sem03-ints-floats/sem3_floats.ipynb to markdown
[NbConvertApp] Writing 2857 bytes to ./sem03-ints-floats/sem3_floats.md
[NbConvertApp] Converting notebook ./sem03-ints-floa

In [ ]:
" ".join(['jupyter', 'nbconvert', './sem04-asm-arm/arm.ipynb', '--to', 'pdf', '--output', 'arm'])

In [ ]:
# !jupyter nbconvert ./sem04-asm-arm/arm.ipynb --to pdf --output arm


In [ ]:
for sfx in [".ipynb", ".md"]:
    for fname in glob.glob("./sem*/*" + sfx):
        with open(fname, "r") as f:
            r = f.read()
        for b in ["\x00", "\x1B", "\x08"]:
            r = r.replace(b, "")
        if sfx == ".md":
            r = r.replace("```python\n%%cpp", "```cpp\n%%cpp")
        with open(fname, "w") as f:
            f.write(r)
        get_ipython().system("dos2unix {}".format(fname))

In [ ]:
for subdir in glob.glob("./sem*"):
    for sfx in [".ipynb", ".md", ".c", ".cpp"]:
        get_ipython().system("git add {}/*{}".format(os.path.basename(subdir), sfx))
get_ipython().system("git add -u")
get_ipython().system("git commit -m 'yet another update'")
get_ipython().system("git push origin master")

In [ ]:
hex(ord('a'))

In [ ]:
"\x61"